<a href="https://colab.research.google.com/github/JanEggers-hr/chatgpt-playground/blob/main/offene_fragen_auswerten_mit_ki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Offene Fragen auswerten mit KI

**Antworten auf offene Fragen kategorisieren und sortieren**

Dieses Colab nimmt die Antworten aus Freitext-Feldern - und versucht sie mit KI-Hilfe zu kategorisieren.

## Was man braucht
- Ein Google-Konto (oder eine lokale juPyter-Installation), um den Code ausführen zu können
- Ein API-Token der OpenAI-KI
- Eine Excel-Tabelle mit den Freitext-Antworten

### Das Format der Excel-Tabelle

| ID | Text Frage 1 | Text Frage 2 | Text Frage 3 | ... |
|---:|:------------:|:-------------:|:----:|:---|
| 1 | Antwort | Antwort | |
| 3 |  | Antwort | | |
| 6 | Antwort |  | Antwort |

## Wie es abläuft

- **Block 1** starten: Lädt die OpenAI-Bibliotheken, um mit der KI sprechen zu können, überprüft, welche KI-Modelle genutzt werden können, und legt all die "Prompts" fest - die Anweisungen an die KI.
- **Block 2** verbindet das Programm mit dem "Google Drive" des Nutzers, lädt eine neue Excel-Datei hoch - oder, wenn keine hochgeladen wird, nimmt die letzte aus dem Ordner ```evaluiere-umfragen-tabelle``` auf dem Google Drive/Meine Ablage - und lässt mich eine Spalte der Tabelle auswählen.
- **Block 3** analysiert die Antworten und schlägt eine vorgegebene Anzahl von Kategorien vor - und gibt dann

## Für die Profis...

...befinden sich ganz am Ende des Dokuments Informationen zu Versionsnummer und angedachten Erweiterungen.

...gibt es die Möglichkeit, in den Code zu schauen - das lohnt sich insbersondere bei Block 1: Am Ende des KI-Code-Blocks sind alle **Prompts** zu finden - die Anweisungen an die KI. Dort nach Bedarf anpassen, wenn die KI nicht tut, was sie soll!

...Informationen zu den verwendeten Temperaturen: Bei der Analyse-Phase kann man sie einstellen. Beim Kategorisierungs-Schritt steht sie auf 0 - um möglichst reproduzierbare Ergebnisse zu haben. Ganz ausgeschaltet ist der Zufall trotzdem nicht.


In [ ]:
#@title Block 1: KI einrichten
import requests
import json
import math
import markdown

# ipywidgets ist schon installiert
import ipywidgets as widgets
from IPython.display import display

# Modelle und Kosten definieren
# Kosten in US-Dollar je 1000 Tokens
# Könnte die Modelle auch über die API holen, aber so kann ich die Kosten
# mitgeben. Entsprechend OpenAI-Preisliste Dezember 2023. https://openai.com/pricing
# Output ist inzwischen doppelt so teuer wie Input. Hier werden die Output-
# Preise zur Berechnung genutzt.
models_token_info = {
          'gpt-4-0125-preview': {
                                        'output_price': 0.03,
                                        'input_price': 0.01,
                                        'max_tokens': 128000,
                                        'info': 'Standard-GPT4'
                                      },
          'gpt-3.5-turbo-0125': {
                                        'output_price': 0.0015,
                                        'input_price': 0.0005,
                                        'max_tokens': 16385,
                                        'info': 'Standard-GPT3.5'
                                      },
                    'gpt-4': {
                                        'output_price': 0.06,
                                        'input_price': 0.03,
                                        'max_tokens': 16384,
                                        'info': 'Teuerere GPT4-Variante (2x)'
                                      },
                    'gpt-3.5-turbo-instruct': {
                                        'output_price': 0.002,
                                        'input_price': 0.0015,
                                        'max_tokens': 4096,
                                        'info': 'Auf Instruktionen optimierte GPT3.5-Variante mit kleinem Kontext'
                                      },

}



# Tokenizer Tiktoken einbinden
!pip install -q tiktoken
import tiktoken
print("Tokenizer tiktoken geladen.")

# OpenAI-API-Library einbinden
!pip install -q openai
from openai import OpenAI
print("OpenAI-API-Library geladen.")

##### Der eigentliche Code! #####

from getpass import getpass
from google.colab import userdata

key_needed = True
try:
  ai_key_name = userdata.get('openai')
  ai_client = OpenAI(api_key = userdata.get(ai_key_name))
except:
  print("OpenAI-Key benötigt")
else:
  print("*** API-Key gültig, KI einsatzbereit! ***")
  key_needed = False

while key_needed:
    try:
        # Testweise Modelle abfragen
        ai_client = OpenAI(api_key = getpass("OpenAI-API-Key eingeben: "))
        models = ai_client.models.list()
        # Returns a list of model objects
        # Erfolg?
        print("API-Key gültig!")
        print("Wenn du willst, kannst du den Key als Colab-Secret hinterlegen")
        print("(unter dem Schlüsselchen am linken Rand). Er wird dort sicher und")
        print("nur für dich in deinem Account gespeichert.\n")
        print("Zwei Secrets müssen angelegt werden:")
        print("- ein Key mit dem OpenAI-API-Key (z.B.: 'openaikey1': 'sk-abcd123...')")
        print("- ein Key 'openai', der den Namen des Secrets mit dem Key bekommt ")
        print("  (im Beispiel: 'openai': 'openaikey1')")
        print("\nDu darfst den Key aber gern auch jedesmal neu eingeben.")
        key_needed = False
    except Exception as e:
        print("Fehler bei Abfrage; ist der API-Key möglicherweise ungültig?", e)

previous_messages = []
spent_tokens = 0        # Wie viele Tokens wurden bisher über die API abgefragt?
spent_dollars = 0.00    # Zu welchem Preis?
tokens_input = 0
tokens_output = 0


# Define the HTML and JavaScript code for the spinner animation
spinner_html = """
<div class="loader"></div>
<style>
.loader {
  border: 8px solid #f3f3f3;
  border-top: 8px solid #3498db;
  border-radius: 50%;
  width: 25px;
  height: 25px;
  animation: spin 2s linear infinite;
  margin: 20px auto;
}

@keyframes spin {
  0% { transform: rotate(0deg); }
  100% { transform: rotate(360deg); }
}
</style>
"""


# Funktion, die das Sprachmodell direkt anzapft (für Preprocessing und Kategorienbildung)
# Braucht: Systemprompt, Beispiele (als User-Assistant-Dialog), Modell
# Gibt zurück: einen String
# Updated die verbrauchten Tokens.

# Hilfsfunktion: Token berechnen
def calculate_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    # cl100k_base ist der Tokenizer für Davinci, GPT-3 und GPT-4
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

def output_pricing(tokens,model):
    price = models_token_info.get(model)['output_price']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def input_pricing(tokens,model):
    price = models_token_info.get(model)['input_price']
    # Kosten in Dollar zurückgeben
    return(tokens * price / 1000)

def gpt(g_system,g_previous_messages,g_prompt,g_model,temperature = 0,json=False,max_tokens=None):
    # Systemprompt und few-shots zusammenbinden
    #
    global spent_tokens
    global spent_dollars
    if json==True:
        m_type = {"type": "json_object"}
    else:
        m_type = {"type": "text"}

    prompts = [
            {"role": "system", "content": g_system},
            *g_previous_messages,
            {"role": "user", "content": g_prompt},
        ]
    response = ai_client.chat.completions.create(
        messages=prompts,
        model=g_model,
        # max_tokens=max_tokens,
        n=1,
        response_format= m_type,
        stream = False,
        temperature=temperature,
    )
    # Anzahl verbrauchtert Tokens anpassen
    spent_tokens += response.usage.total_tokens
    tokens_input = response.usage.prompt_tokens
    tokens_output =  response.usage.completion_tokens
    spent_dollars += output_pricing(tokens_output,model) + input_pricing(tokens_input,model)
    token_usage_text = f'<b>Verbrauchte Token:</b> {spent_tokens} ($ {spent_dollars:.3f}) '
    return(response.choices[0].message.content)

### Liste der verfügbaren Modelle anpassen ###
all_models = ai_client.models.list()
all_model_names = [model.id for model in all_models.data]
for key in list(models_token_info.keys()):
    if key not in all_model_names:
        del models_token_info[key]
        print(f"OpenAI-Modell {key} für diesen API-Key leider nicht verfügbar")
    else:
        print(f"Modell {key} verfügbar: {models_token_info[key]['info']}")

##############################################################################
##############################################################################
##############################################################################
# PROMPTS - die Anweisungen an die KI
#
# Hier kannst du anpassen, welche Anweisungen die KI bekommt.

##############################################################################
# Das Analyse-Prompt:
# Im ersten KI-Schritt schaut das Sprachmodell auf eine Stichprobe von Antworten
# und versucht, daraus Kategorien vorzuschlagen.
#
# Als Funktion, um zur Laufzeit die Anzahl der gewünschten Kategorien und die Frage
# vorzugeben.

def analyse_prompt(kategorien,frage):
    analyse_system = f"""Du bist Meinungsforscher.
    Dies sind Antworten auf die Frage: '{frage}'
    Fasse bitte die Antworten in {kategorien} Kategorien zusammen. Die Kategorien
    können Inhalte benennen. Sie sollten eine Kategorie 'Sonstiges'
    enthalten und eine Kategorie, die Antworten markiert, die an der Frage vorbeigehen.
    Beschreibe die Kategorien ausführlich und trennscharf. Liste die ausführlichen
    Beschreibungen der Kategorien fortlaufend nummeriert auf. Erzeuge ein JSON mit
    Kategorie, Beschreibung der Form: {{'ID': {{'Kategorie': text, 'Beschreibung': text}} }}.
    """
    return(analyse_system)

##############################################################################
# Preprocess: Leere und unsinnige Antworten markieren, um sie zu filtern
#
# WIRD DERZEIT GAR NICHT GENUTZT, BUT HEY!

def preprocess_prompt(p,frage):
    preprocess_system = f"Du bist Meinungsforscher. \
    Dies sind Antworten auf die Frage: '{frage}' \
    Nenne die Nummern der Antworten, die keine Angaben \
    machen oder nicht zu verstehen sind. Antworte mit einem JSON."
    sample_input="""1073 |
    1099 | wieß nicht
    1101 | keine Meinung"""
    sample_output="{'1073': True, '1099': True, '1101': True }"
    preprocess_samples = [
              {"role": "user", "content": sample_input},
              {"role": "assistant", "content": sample_output},
          ]
    p_filter_text = gpt(preprocess_system,preprocess_samples,p,model,json=True)
    p_filters = [int(num) for num in re.findall(r'\d+', p_filter_text)]
    return(p_filters)

##############################################################################
# Das Prüf-Prompt:
#
# Die KI soll bewerten, wie gut brauchbar die Kategorien sind. Hat derzeit nur
# Informationscharakter.

def feedback_prompt(frage):
  pruef_prompt = f"""Du bist Meinungsforscher. Du beurteilst Kategorien, \
  die Antworten auf diese Frage zusammenfassen: \
  '{frage}' Die Kategorien sollen eindeutig, \
  klar definiert und trennscharf sein. \n\
  Sichte die vorgeschlagenen Kategorien und bewerte jede Kategorie in Hinblick \
  darauf, ob sie klar definiert und eindeutig ist, auf einer \
  Skala von 0 (unzureichend) bis 10 (perfekt).
  Sichte dann die Bewertungen. Bei Werten von 6 oder weniger \
  mach einen Vorschlag zur Verbesserung, zum Beispiel eine Aufteilung in \
  eindeutige Kategorien."""
  return pruef_prompt

##############################################################################
# Die Kategorisierungs-Funktion:
#
# Ordnet die übergebenen Antworten zur Frage in die Kategorien ein,
# orientiert sich dabei an vorgegebenen Beispielen ("few-shot").
# Gibt ein JSON zurück.

def kategorisiere(textblock,
                  kat_text,
                  samples,
                  antwort,
                  model,
                  frage):
  kat_system = f"""
  Du bist Meinungsforscher.
  Du bekommst Antworten auf die Frage: '{frage}'.
  Prüfe für jede Antwort, ob sie den folgenden Kategorien entspricht.
  ###
  {kat_text}
  ###
  Untersuche jede Antwort und beantworte
  für jede Kategorie, ob die Antwort in diese Kategorie passt.
  Antworte mit einem JSON mit dem Index ID und einer Liste für alle Kategorien
  mit Elementen der Form: {{'A': True}} oder {{'A': False}}.
  """
  kat_sample = [
      {"role": "user", "content": samples},
       {"role": "assistant", "content": antwort},

  ]
#  print(kat_system)
#  print(kat_sample)
#  print(textblock)
  result = gpt(kat_system,
               kat_sample,
               textblock,
               g_model = model,
               # Temperatur 0 - wir wollen Reproduzierbarkeit!
               temperature=0,
               json= True)
#  # Gib einen String zurück
#  print("-------------")
#  print(result)
  return result

Dauert eine Weile, bis alle Bibliotheken geladen sind - aber diesen Schritt braucht es, damit wir die KI einsetzen können. Wenn das Notebook die Verbindung zum Colab-Server verloren hat, muss man bei dieser Zelle noch einmal anfangen und alles neu ausführen - Colab merkt sich keine Zwischenstände, wenn die "Laufzeit" (der virtuelle Rechner) stoppt.

Wer etwas an den KI-"Prompts" ändern will: Sie finden sich oben im Block 1 ganz am Ende des Codes! Auch für Nicht-Programmierer - Änderungen einfach an den roten Texten zwischen den Dreifach-Anführungszeichen-Zeilen ``` """ ``` durchführen. (Alles Grüne mit einem ``` # ``` vorneweg sind Kommentare, die der Computer ignoriert.)

## Jetzt: Daten laden!

Google Drive ist der Cloud-Speicherplatz, den jedes Google-Konto nutzen kann - wie eine Dropbox oder Nextcloud.

Hochgeladene .XLSX-Dateien legt das Programm im Google-Drive Verzeichnis ```gdrive/MyDrive/evaluiere-umfragen-tabelle``` ab. (Von dort holt es sich auch die jeweils neueste Datei, wenn man keine Datei hochlädt.)


In [ ]:
#@title Block 2: Daten laden

from google.colab import files, drive
# Connect to Google Drive to export data
import os
import io
import pandas as pd
import glob
import json
import markdown
from datetime import datetime

drive.mount('/content/gdrive')

# Create output directory
output_dir = "/content/gdrive/MyDrive/evaluiere-umfragen-tabelle"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
if not os.path.exists(output_dir + "/ergebnisse"):
    os.mkdir(output_dir + "/ergebnisse")

os.chdir(output_dir)

def upload_and_load_excel():
    uploaded = files.upload()
    if uploaded:
        file_name = next(iter(uploaded))
        path = io.BytesIO(uploaded[file_name])
        # Laden der Excel-Datei in ein Pandas DataFrame
        df = pd.read_excel(path, index_col=0)  # Hier wird angenommen, dass die erste Spalte als Index dient
        return df
    else:
        # Letzte xlsx-Datei im Pfad
        xlsx_files = glob.glob(os.path.join("./", '*.xlsx'))
        most_recent_file = max(xlsx_files, key=os.path.getctime)
        df = pd.read_excel(most_recent_file, index_col=0)
        return df

df = upload_and_load_excel()
if df is not None:
    dropdown_spalte = widgets.Dropdown(
        options = df.columns,
        description = 'Spalte:',
        disabled = False,
        layout = widgets.Layout(width="50%")
    )
    text_spalte = widgets.HTML(
        value=""
    )
    def d_eventhandler(change):
      global df_clean
      global column_name
      column_name = dropdown_spalte.value
      series = df[column_name]
      df_clean = series.dropna()  # Entfernen von leeren Zellen
      text_spalte.value = f"enthält {len(df_clean)} Zeilen"
    dropdown_spalte.observe(d_eventhandler,'value')
    # 1x ausführen, um erste Spalte anzuzeigen
    d_eventhandler(0)
    hbox_spalte = widgets.HBox([dropdown_spalte, text_spalte])
    display(hbox_spalte)

Wenn ich eine Spalte auswähle, kriege ich angezeigt, wie viele Zeilen sie enthält - nachdem die leeren Antworten ausgefiltert wurden.

## Analyse: Die KI Kategorien vorschlagen und feedbacken lassen

Im weiteren arbeitet das Programm **nur mit den Daten aus der hier ausgewählten Spalte** - wer die anderen Spalten analysiert haben will, muss ab Block 3 nochmal durchlaufen lassen!

* Ein Klick auf den Knopf **Analyse** zieht eine Stichprobe aus den Antworten - bis zu 500 - und lässt die KI die vorgegebene Anzahl Kategorien vorschlagen.
* Ein Klick auf den Knopf **Feedback** lässt die KI prüfen, für wie trennscharf und präzise sie die vorgeschlagenen Kategorien hält.

Dabei kann man einstellen:
* Das Modell - die stärkere KI (GPT4-...) oder die schnellere (GPT3.5-turbo...)?
* Die Temperatur - wie viel Zufall lassen wir zu; wie unvorhersehbar soll das Sprachmodell antworten?

Die vorgeschlagenen Kategorien kann man direkt hier anpassen - oder etwas bequemer im nächsten Schritt.

Wer mag, kann Kategorien auch selbst eintragen, ohne KI - die allerdings als "Markdown"-Tabelle formatiert sein sollten, also so:
```
| ID | Kategorie | Beschreibung |
|----|-----------|--------------|
| 1  | Sonstiges | Alles, was nicht klar einer der anderen Kategorien zuzuordnen ist |
```
(Das hier kann man als Kopiervorlage nutzen!)

In [ ]:
#@title Block 3: Analyse und Einteilung
#

# Debugging-View
debug_view = widgets.Output(layout={'border': '1px solid black'})
# Beispielabfrage
# @debug_view.capture(clear_output=True)


import markdown

kategorien = 10
max_samples = 500
if len(df_clean) > max_samples:
  max_samples = len(df_clean)

# Das Interface

# Konstanten
wid90 = widgets.Layout(width="90%")
wid80 = widgets.Layout(width="80%")
wid50 = widgets.Layout(width="50%")
wid40 = widgets.Layout(width="40%")
wid30 = widgets.Layout(width="30%")

# Slider für die Temperatur (Default: 0.3)
slider_temperatur = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1.5,
    step=0.1,
    description='Temperatur:',
    orientation='horizontal',
    readout=True,
    layout = wid50
)


# Slider für die Anzahl der Kategorien (Default: 10)
slider_kategorien = widgets.IntSlider(
    value=10,
    min=5,
    max=20,
    description='Kategorien:',
    orientation='horizontal',
    readout=True,
    # Weite begrenzen, damit der Readout mit angezeigt wird
    layout=wid90
)

slider_samples = widgets.IntSlider(
    value=100,
    min=20,
    max=max_samples,
    description='Stichprobe:',
    orientation='horizontal',
    readout=True,
    layout = wid90
)

textbox_max_tokens = widgets.Text(
    value='0',
    placeholder='0',
    description='Max. Token:',
)

text_tokens = widgets.HTML(
    value = f'<b>Samples in Token</b>: 0 '
)

dropdown_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_token_info.keys()),
    value=list(models_token_info.keys())[0],
    description='Modell:',
)

area_system = widgets.Textarea(
    value = analyse_prompt(kategorien,column_name),
    rows=10,
    description = 'Kategorien:'
)


area_kategorien = widgets.Textarea(
    value = '(noch keine Kategorien)',
    rows=15,
    description = ''
)

html_samples_tokens = widgets.HTML(
    value = ''
)

button_analyse = widgets.Button(
    description='KI-Vorschlag',
    tooltip = 'Die KI auf Basis der Stichprobe Kategorien-Vorschläge machen lassen',
    layout = wid80
)

button_feedback = widgets.Button(
    description='KI-Feedback',
    tooltip = 'Fragt die KI, wie es die derzeitigen Kategorien einschätzt',
    layout = wid80
)

html_status = widgets.HTML(value ="")

# Definiere Output - um später das Tab neu löschen zu können
output = widgets.Output()



stoptokens=""

# Funktion wird bei Veränderung ausgeführt
def update_params(change):
    global temperature
    global max_tokens
    global system_prompt
    global model
    global stoptokens
    global best_of
    global kategorien
    global samples_n
    global prepare_prompt
    global token_limit_reached
    column_name = dropdown_spalte.value
    # Token-Obergrenze umrechnen
    try:
        max_tokens = int(textbox_max_tokens.value)
        if max_tokens == 0:
            max_tokens = None
    except ValueError:
        max_tokens = None
    textbox_max_tokens.value = f'{max_tokens}'
    system_prompt = area_kategorien.value
    model = dropdown_model.value
    if (stoptokens == ""):
      stoptokens = None
    temperatur = slider_temperatur.value
    kategorien = slider_kategorien.value
    area_system.value = analyse_prompt(kategorien,column_name)
    samples_n = slider_samples.value
    if samples_n > len(df_clean):
        samples_n = len(df_clean)
        slider_samples.max = samples_n

    prepare_prompt = df_clean.sample(n=samples_n).to_markdown()
    sample_tokens = calculate_tokens(prepare_prompt)
    token_limit_reached = (sample_tokens > models_token_info.get(model)['max_tokens'] * 0.9)
    if token_limit_reached:
        text_tokens.value = f'<b style="color:red">Samples in Token: {sample_tokens} - ÜBER DEM LIMIT</b>'
    else:
        text_tokens.value = f'<b>Samples in Token: {sample_tokens} </b>'


########################

# Verbinde die Widgets mit der Funktion zur Verarbeitung der Werte
slider_kategorien.observe(update_params, 'value')
slider_samples.observe(update_params, 'value')
# Datensatz in ein JSON verwandeln


# Auswertungsfunktionen für die Kategorien:
# - Erster Durchlauf: Markiere die Nicht-Antworten
kategorien = 10

@debug_view.capture(clear_output=True)
def prepare_categories(p):
    global kategorien
    global prepare_prompt
    global kat_json
    # Spalte
    column_name = dropdown_spalte.value
    html_samples_tokens.value = f"Verbrauchte Tokens: {spent_tokens}, {spent_dollars:.2f} $"
    prepare_prompt = ""
    prepare_samples = []
    prepare_prompt = p.to_json(orient="index")
    prepare_system = analyse_prompt(kategorien,column_name)
    print(prepare_system)
    print(prepare_prompt)
    html_status.value = spinner_html + f"<b>Starte Analyse</b> mit Prompt: <small>{prepare_prompt[:100]}</small>"
    # Spinner einschalten
    model = dropdown_model.value
    kategorien_vorschlaege = gpt(prepare_system,
                                prepare_samples,prepare_prompt,model,json=True)
    # Vorschlags-String in Tabelle umwandeln und die anzeigen
    # Erst ein dict
    print("---Output---")
    print(kategorien_vorschlaege)
    kat_json = json.loads(kategorien_vorschlaege)
    # ...dann daraus ein Dataframe und das in Markdown
    area_kategorien.value = pd.DataFrame.from_dict(kat_json,orient="index").to_markdown()
    html_samples_tokens.value = f"Verbrauchte Tokens: {spent_tokens}, {spent_dollars:.2f} $"
    html_status.value = "Kategorien in Tabelle umgewandelt"

#    html_status.close()
#    area_kategorien.display()

def generiere_kategorien(change):
    global df_clean
    global frage
    # Wird nur ausgeführt, wenn Token-Limit noch nicht erreicht ist
    if not (token_limit_reached):
    # Stichprobe von n Samples aus den Antworten ziehen (stellt man mit dem
    # Slider ein)
        sample_df = df_clean.sample(n=samples_n)
        prepare_categories(sample_df)

def generiere_feedback(change):
    if (area_kategorien.value == "") or area_kategorien.value == "(noch keine Kategorien)":
      html_status.value = "<i>Keine Kategorien angelegt!</i>"
      return False
    html_status.value = "<b>Überprüfe Kategorien...</b>"+spinner_html
    model=list(models_token_info.keys())[0]
    # Hole Feedback von der KI: Sind die Kategorien trennscharf und
    # hinreichend bestimmt?
    # Gibt derzeit nur einen Text zurück - für die Zukunft: Vorschläge
    # im Editor markieren.
    pruefe_vorschlag = gpt(feedback_prompt(dropdown_spalte.value),
     [],area_kategorien.value,
                            g_model=model,temperature = 0.5,json=False)
    html_status.value = "<h2>GPT-4 schätzt die Kategorien so ein: </h2>" + markdown.markdown(pruefe_vorschlag, extensions=['tables'])
    return pruefe_vorschlag

# Bereite die Kategorien vor.
button_analyse.on_click(generiere_kategorien)
button_feedback.on_click(generiere_feedback)

# Die Einstellungs-Widgets anzeigen
# Setzt die globalen Variablen temperature, system_prompt, api_key, model, stoptokens
display(dropdown_model)
# Widget mit der Anzahl an Kategorien anzeigen (Default = 10)

update_params(0)
vbox_analyse = widgets.VBox([slider_kategorien,
        slider_samples,
        button_analyse,
        button_feedback],
                            layout=wid40)
hbox_kategorien = widgets.HBox([vbox_analyse, area_kategorien])
display(slider_temperatur)
display(text_tokens)
display(hbox_spalte)
display(html_samples_tokens)


# Ergebnistext über volle Breite
area_kategorien.layout.width = '100%'
display(html_status)
display(hbox_kategorien)


**Einverstanden mit den Kategorien?** Die Anzahl und die Definition wie gewünscht oben abändern; dabei bitte keine leeren Zeilen einfügen bzw. Neudefinitionen mit laufender Nummer am Ende ergänzen

Wenn alles so passt: Weiter mit dem nächsten Block 4!

## Kategorien testen und anpassen - mit Beispielen

Im nächsten Schritt probieren wir an einigen Beispielen aus, wie die KI mit diesen Kategorien arbeitet. Die Kategorisierungen können wir von Hand anpassen (und auch die Kategorie-Definitionen können wir hier noch ändern, um zusätzliche Kategorien ergänzen oder Kategorien löschen.)

Warum die Mühe? Sprachmodelle arbeiten besser, wenn man ihnen an einigen Beispielen zeigt, wie sie arbeiten sollen - "few-shot learning" nennen das KI-Leute. Die Beispiele und das gewünschte Ergebnis werden bei der Kategorisierung mit übergeben.

In [ ]:
#@title Block 4: Kategorien testen und Beispiel-Kategorisierungen verbessern
################################################################################
################################################################################
################################################################################

import re

chk_samples_n = 5
slider_beispiele = widgets.IntSlider(
    value=5,
    min=3,
    max=10,
    description='Beispiele:',
    orientation='horizontal',
    readout=True,
    layout = wid50
)

def update_chk_samples(v):
  global chk_samples_n
  global chk_samples_df
  global tab_samples
  chk_samples_n = slider_beispiele.value
  # Neue Samples ziehen
  chk_samples_df = df_clean.sample(n=chk_samples_n)
  generate_raster()
  generate_samples_list()
  tab_samples = create_tabs()
  output.clear_output()
  with output:
    display(tab_samples)



slider_beispiele.observe(update_chk_samples)
# Globale Variable kategorien enthält die Anzahl der Kategorien laut Schieberegler,
# muss auf tatsächliche Anzahl gesetzt werden
kategorien_text = area_kategorien.value

# Splitte die tabelle in einzelne Teilen; Trennzeile raus.
# Den String mit dem Markdown-Code in Zeilen zerlegen
# kat_df enthält dann die Kategorien mit einer Kategorie-ID
zeilen = [line.strip() for line in kategorien_text.strip().split("\n") if line.strip() and not all(c in '-|:' or c.isspace() for c in line)]
kategorien = len(zeilen)-1

def kat_strip(string):
  strings = string.split("|")
  strings2 = [s.strip() for s in strings[2:]]
  return ": ".join(strings2)

kat_df = pd.DataFrame([kat_strip(string) for string in zeilen[1:]])

############## POINT OF NO RETURN ################
# Ab hier werden die Kategorien in dieser Zelle bearbeitet; jedes Mal,
# wenn eine Kategorie gelöscht oder neu geschaffen wird, wird die Tabelle neu aufgebaut

# Slider nochmal anpassen - falls man die Kategorien nochmal nachtunen will
slider_kategorien.value = kategorien
# Alte GUI-Elemente weg
slider_kategorien.close()
area_kategorien.close()
slider_temperatur.close()
slider_samples.close()
hbox_kategorien.close()

# Definiere Output - um später das Tab neu löschen zu können
output = widgets.Output()


# Beispiele ziehen
chk_samples_df = df_clean.sample(n=chk_samples_n)


print("In diesem Abschnitt haben Sie die Möglichkeit, der Maschine beim Einordnen der Antworten zu helfen.")
print()
print("Wir haben ein paar Zufalls-Antworten gezogen. Sie können sie entweder von Hand kategorisieren, ")
print("wie Sie es mit den Kategorien tun würden, oder auf 'Testen' klicken - dann versucht die KI,")
print("die Beispiele einzuordnen. Klicken Sie sich durch die Beispiel-Tabs und korrigieren Sie nach.")
print("Auch die Definitionen lassen sich noch verbessern. Wenn Sie zufrieden sind, können Sie weitergehen.")

display(slider_beispiele)
html_status2 = widgets.HTML("")

# Das Auswahlraster generieren
# Arbeitet mit den globalen Variablen:
# - labels (Liste mit Textboxen für die Kategorien)
# - raster (Liste von Listen bzw. Array mit den Checkboxen)


# Erst mal: Die Labels, für jede Kategorie eins.
# Dienen ab hier auch als Speicher für die Kategorien.
def generate_labels():
    global labels
    global kategorien
    kategorien = len(kat_df)
    labels = [widgets.Text(value = kat_df[0][i],layout=wid50) for i in range(kategorien)]


# Dann das Raster: Für jedes Sample eine Zeile, für jede Spalte eine Kategorie.
def generate_raster():
    global raster
    raster = []
    raster = [[widgets.Checkbox(value=False,
                                description="trifft zu") for j in range(kategorien)] for i in range(chk_samples_n)]

# Zu kategorisierende Texte aktualisieren
def generate_samples_list():
  global html_samples_list
  global chk_samples_df
  html_samples_list = [widgets.HTML(f"<h2>Zu kategorisierender Text:</h2> <b><i>>>{s}<<</i></b>")for s in chk_samples_df]
  return html_samples_list


# Jetzt die Tabs generieren und anzeigen: VBox von HBoxes.
def make_all_tabs():
  # Für jedes Sample ein Child-Element anlegen:
  # - Erste Zeile: Label mit dem Sample-Text
  # - Dann für jede Kategorie eine HBox mit Kategorie und Checkbox
  global all_tabs
  all_tabs = []
  global html_samples_list
  for i in range(chk_samples_n):
    # Liste anlegen
    rows = [html_samples_list[i]]
    for j in range(kategorien):
      rows.append(widgets.HBox([labels[j],raster[i][j]]))
    all_tabs.append(widgets.VBox(rows))

def create_tabs():
  make_all_tabs()
  # Jetzt das Tab-Element generieren und anzeigen
  tab = widgets.Tab(
      children = all_tabs,
      titles = [f"Beispiel {i+1}" for i in range(chk_samples_n)]
  )
  for i in range(chk_samples_n):
    tab.set_title(i, f"Beispiel {i+1}")
  return tab

def generate_all():
  generate_samples_list()
  generate_labels()
  generate_raster()
  tab = create_tabs()
  return tab

def check_empty(raster):
  for row in raster:
      # Iterate through each checkbox in the row
      for checkbox in row:
          # Check if the checkbox is checked (True)
          if checkbox.value:
              # If any checkbox is checked, return False
              return False
  # If all checkboxes are unchecked, return True
  return True

# Hilfsfunktion: Alle Checkboxen auf False setzen
def reset_check(v):
  for row in raster:
    for checkbox in row:
      checkbox.value = False

# Hilfsfunktion: eine Kategorie aus dem Checkboxen-Raster löschen
def raster_remove(kat):
    global raster
    for row in raster:
      del row[kat]

# Hilfsfunktion: Textboxen in kat_df umkopieren; labels anpassen
def update_kat():
  global kat_df
  global labels
  kat_liste = [kat.value for kat in labels]
  kat_df = pd.DataFrame(kat_liste)

# Hilfsfunktion: label-Textboxen in Kategorien-String-Liste
def chk_kat_new():
  kat_new = ""
  i = 0
  for l in labels:
    kat_new = kat_new + chr(65+i) + ". | " + l.value + "\n"
    i = i + 1 # All-Time Classic!
  return kat_new

# Hilfsfunktion: Aus raster ein Dictionary als fewshot-Beispiel machen
def create_fewshot(raster):
      # JSON mit den Antworten generieren
    d = {}
    # Reihen haben wir immer.
    # Steppe durch die Indices des Dict.
    for y in range(len(raster)):
      row_list = []
      for kat in range(len(raster[y])):
        kat_key = chr(kat + 65) # 'A' ff.
        row_list.append({kat_key: raster[y][kat].value})
      d[f"{y}"] = row_list
    return d

# Hilfsfunktion: JSON in Raster verwandeln
# Nimmt einen JSON-String.
# Gibt ein df mit Bool-Werten und den Indizes der Zeilen zurück
def rasterize(json_str1):
    d = json.loads(json_str1)
    # neues Dataframe anlegen - Spalten A, B, C... (für Kategorien)
    global labels
    column_names = [chr(k+ord("A")) for k in range(len(labels))]
    df = pd.DataFrame(columns = column_names).astype(bool)
    df.index.name = "ID"
    # Durch alle Keys (ID) des Root-Directorys gehen.
    # Sei json_str ein JSON der Form: { '1': [T,F,T], '2': [F,T,T]}
    # oder { '1': [{'A': T}, {'B':T}] }
    # oder { '1': [{'A': T, 'B': F}], '2': {'A': F, 'B': T} }
    # oder { '1': [F, T, T, F, T, T, F]}
    # oder { '1':{'A': F, 'B': T, 'C':F}, '2': {'A': F, 'B': T} }
    #
    for idx in d.keys():
    # Zusatz: Ignoriere leere Keys.
    # Kam immer wieder vor, dass das Sprachmodell einen leeren Key ausspuckt.
      if idx != "":
        zeile = d[idx]
        # Isch scho Lischde?
        if isinstance(zeile,list):
          # Prüfe erstes Element: Dictionary?
          if isinstance(zeile[0],dict):
            # JSON flach machen - Liste in einzelne Dict-Einträge konvertieren
            flattened_dict = {list(d.keys())[0]: list(d.values())[0] for d in zeile}
            new_df = pd.DataFrame(flattened_dict,index=[idx]).astype(bool)
            # Oder einzelnes Element: dann einfach die Liste zuweisen
          else:
            new_df = pd.DataFrame([zeile], columns = column_names, index = [idx]).astype(bool)
          df = pd.concat([df,new_df],ignore_index=False)
        # Die Zeile kann auch ein dict sein statt einer Liste
        elif isinstance(zeile,dict):
          new_df = pd.DataFrame(zeile, index = [idx]).astype(bool)
          df = pd.concat([df,new_df],ignore_index=False)
        else: # Einzelelement?
          print(zeile,"kann nicht verarbeitet werden?")
    # Zur Sicherheit: NaN-Werte - kein Eintrag - werden durch False ersetzt
    df.fillna(False, inplace=True)
    return df


# Beispielabfrage
# Druckt viele Debug-Informationen, die aber alle im Objekt debug_view landen.
# Wenn man Debug-Daten braucht, kann man es sich über debug_view anzeigen lassen.
@debug_view.capture(clear_output=True)
def evaluate(value):
  global chk_samples_df
  global tab_samples
  global html_samples_list
  chk_samples_df.name="Antwort"
  global beispielantwort
  # Die Checkboxen auslesen und in ein Beispiel-JSON packen,
  # außer sie sind alle leer.
  output.clear_output()
  if check_empty(raster):
    beispiele = ""
    beispielantwort = ""
  else:
    # Es gibt schon Bewertungen: übergib die existierenden als Beispiel und die
    # existierenden Bewertungen als Beispielantwort - als Few Shot.
    # Dann n neue Beispiele ziehen und existierende mit der ANtwort übergeben
    # Erst mal: Index 0, 1, 2... (passend zu den Few-Shots)
    chk_samples_df.reset_index(drop=True, inplace=True)
    beispiele = chk_samples_df.to_markdown()
    # JSON mit den Antworten aus raster generieren
    d = create_fewshot(raster)
    beispielantwort = json.dumps(d)
    # Beispiel und passende Antworten sind übergeben; jetzt neue Samples ziehen
    chk_samples_df = df_clean.sample(n=chk_samples_n)
    chk_samples_df.index.name="ID"
    # Index 0, 1, 2, 3, 4 - für stimmige Beispiele
    chk_samples_df.name = "Antwort"
    # Liste mit den neuen Beispielen als HTML-Titel
    generate_samples_list()
  # Nochmal die Kategorien anpassen (aus augenblicklichen Labels-Textboxen)
  update_kat()
  # Unseren Spinner nutzen
  html_status2.value="<h2>Evaluiere die Beispiele..."+spinner_html
  print("---Kategorien aus chk_kat_new---")
  print(chk_kat_new())
  print("---Few-Shot---")
  print(beispiele)
  print(beispielantwort)
  print("---Samples---")
  print(chk_samples_df.to_markdown)
  # Modell abfragen
  json_str = kategorisiere(textblock = chk_samples_df.to_markdown(),
                kat_text = chk_kat_new(),
                  samples = beispiele,
                  antwort = beispielantwort,
                  model = dropdown_chk_model.value,
                  frage=column_name)
  print(f"---Input Tokens: {tokens_input} Output Tokens: {tokens_output}---")
  print("---json_str---")
  print(json_str)
  html_status2.value="Update Beispiele..."
  # Aus dem JSON ein Raster von Boolean-Werten
  # Funktion gibt ein df zurück, das
  values = rasterize(json_str).values.tolist()
  # Rüberkopieren auf die Checkboxes
  for y in range(len(values)):
    v = values[y]
    for x in range(len(v)):
      raster[y][x].value = v[x]
  ### Update der Tabs ###
  html_status2.value=f"Beispiel-Kategorien für Frage: {column_name}"
  html_samples_tokens.value = f"Verbrauchte Tokens: {spent_tokens}, {spent_dollars:.2f} $"
  # Update der Tabelle und der Tab-Spaltentexte
  # Eigentlich sollte man die nur neu generieren, aber offensichtlich
  # erneuern sich die HBox- und VBox-Elemente nicht, also nochmal anzeigen
  tab_samples = create_tabs()
  with output:
    display(tab_samples)


dropdown_chk_model = widgets.Dropdown(
    # Nimm die oben definierte Preisliste als Basis
    options=list(models_token_info.keys()),
    value=list(models_token_info.keys())[1],
    description='Modell:',
)

button_chk_evaluate = widgets.Button(
    description = "Testen",
    tooltip = "Lässt die KI die Kategorien auf n Beispiele anwenden"
)
button_chk_reset = widgets.Button(
    description = "Reset",
    tooltip = "Löscht alle Checkboxen; Abfrage ohne Nutzer-Vorgabe"
)
button_neue = widgets.Button(
    description="Neue Kategorie",
    tooltip ="Legt eine neue, leere Kategorie an"
)

button_leere = widgets.Button(
    description="Leere löschen",
                              tooltip = "Leere Kategorien werden gelöscht")

button_neue.style.button_color = "#CCC"
button_leere.style.button_color = "#C99"


def neue_kategorie(v):
  global raster
  global kat_df
  # Das hier legt zwar nur eine Kopie des Pointers auf raster an,
  # aber da generate_raster tatsächlich eine neue Liste generiert,
  # geht sich das trotzdem aus.
  raster_alt = raster
  # Index des Dataframes reparieren und neue Zeile
  kat_df.reset_index(drop=True, inplace=True)
  kat_df.loc[len(kat_df)]=""
  generate_labels()
  # Neues Raster neu anlegen, alte Raster-Werte kopieren
  generate_raster()
  for i in range(len(raster_alt)):
    for j in range(len(raster_alt[i])):
      raster[i][j].value = raster_alt[i][j].value
  global tab_samples
  tab_samples=create_tabs()
  output.clear_output(wait=True)
  with output:
    display(tab_samples)

def leere_kategorien_loeschen(v):
  global tab_samples
  global kategorien
  global kat_df
  update_kat()
  #Liste aus dem Dataframe
  kat_liste = list(kat_df[0])
  # raster_remove
  # Rückwärts zählen, um das Raster bei mehrfachen Kategorie-Löschungen
  # nicht durcheinander zu bringen
  for i in range(len(kat_liste)-1,-1,-1):
    if kat_liste[i] == "":
    # Leere Spalten aus dem Chkbox-Raster löschen
      raster_remove(i)
      kat_df = kat_df.drop(i)
  # Index für Kategorien neu aufbauen
  kat_df.reset_index(drop=True, inplace=True)
  kategorien = len(kat_liste)
  generate_labels()
  tab_samples=create_tabs()
  output.clear_output()
  with output:
    display(tab_samples)


# widgets.tab(children=, titles=)
# Jedes Child als VBox aus: Titelzeile, HBox Prompt und Checkbox.
# Umwandlungs-Funktion Checkboxen-Grid-Values und JSON
# Das für später: Einstellen der Anzahl der Shots
tab_samples = generate_all()

button_chk_evaluate.on_click(evaluate)
# Die Observes für den Text-Update
button_chk_reset.on_click(reset_check)
button_neue.on_click(neue_kategorie)
button_leere.on_click(leere_kategorien_loeschen)

hbox_eval = widgets.HBox([dropdown_chk_model,button_chk_evaluate,button_chk_reset,button_neue, button_leere])
display(html_samples_tokens)
display(hbox_eval)
display(html_status2)
display(output)
with output:
  display(tab_samples)


Wenn alle Beispiele so kategorisiert sind, wie du es dir vorstellst, kann die Auswertung beginnen.

Dabei werden die Antworten in Blöcken der festgelegten Größe an die KI übergeben - das erlaubt eine wesentlich schnellere und kosteneffizientere Verarbeitung. Zu groß sollten die Blöcke nicht werden. Derzeit gehe ich davon aus, dass eine Blockweite von 30 in Ordnung ist - und GPT4 vermutlich auch mit 50 klar kommt. Voreingestellt sind 25.

Während der nächste Block verarbeitet wird, kann man sich anschauen, wie die KI die letzten Antworten bewertet hat.

**Das dauert dann schon ein paar Minuten.** Bitte daran denken: Ein zu lange unbewachtes Colab meldet sich ab! Browsererweiterungen wie [Colab Automatic Clicker](https://addons.mozilla.org/de/firefox/addon/colab-automatic-clicker/) (Firefox) und [Colab Keep Alive](https://chromewebstore.google.com/detail/google-colab-keep-alive/bokldcdphgknojlbfhpbbgkggjfhhaek) (Chrome) helfen.

In [ ]:
#@title Block 5: Auswertung und Datei-Export
from math import floor


slider_blockweite = widgets.IntSlider(
    value=25,
    min=5,
    max=50,
    description='Blockweite:',
    orientation='horizontal',
    readout=True,
    layout = wid50
)

def update_blockweite(v):
  global blockweite
  blockweite = slider_blockweite.value

slider_blockweite.observe(update_blockweite)




# Fortschrittsbalken!
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=len(df_clean),
    description='Fortschritt',
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=wid50
)

# Eigentliche Abfrage der KI - als Funktion, um debuggen zu können
@debug_view.capture(clear_output=False)
def test_block(myblock_df,kat_text,beispiele,beispielantwort):
    print("---Kategorien aus chk_kat_new---")
    print(chk_kat_new())
    print("---Few-Shot---")
    print(beispiele)
    print(beispielantwort)
    print("---Block---")
    print(myblock_df.to_markdown())
    myjson_str = kategorisiere(textblock = myblock_df.to_markdown(),
                  kat_text = kat_text,
                    samples = beispiele,
                    antwort = beispielantwort,
                    model = dropdown_chk_model.value,
                    frage = column_name)
    print("---JSON---")
    print(myjson_str)
    return myjson_str

# Haupt-Funktion
def starte_evaluation(v):
    # Variable als global definieren, um sie ggf. retten zu können
    global auswertung_df
    debug_view.clear_output()
    # Steppe durch die Aussagen, blockweise.
    # Hier geht's los
    blockweite = slider_blockweite.value
    z_dict = {}
    model = dropdown_chk_model.value
    df_clean.index.name="ID"
    df_clean.name="Antwort"
    #display(dropdown_model, button_starten)
    print(f"Modell: {model}, Blockweite: {blockweite}")
    button_start_eval.close()
    slider_blockweite.close()
    html_info = widgets.HTML(value =f"Los geht's! Blöcke zu {blockweite} Kommentaren - Ergebnis wird angezeigt")
    display(html_samples_tokens)
    display(progress)
    display(html_info)
    html_info.value = "Auswertung beginnt... " + spinner_html
    # Die Beispiele final auslesen und nutzen
    # Vorher den Index resetten, damit die Beispiel-Bewertungs-Indizes passen
    chk_samples_df.reset_index(drop=True, inplace=True)
    chk_samples_df.index.name="ID"
    chk_samples_df.name = "Antwort"
    beispiele = chk_samples_df.to_markdown()
    # Hier werden die Raster-Settings in eine Tabelle gelesen, die dann als
    # Beispielantwort angezeigt wird
    d = {}
    for y_str in range(chk_samples_n):
      row_list = [raster[y_str][x].value for x in range(len(raster[y_str]))]
      d[str(y_str)] = row_list
    beispielantwort = json.dumps(d)
    kat_text = chk_kat_new()
    html_info.value = spinner_html
    auswertung_df = pd.DataFrame()
    # Antworten in Blöcke aufteilen
    for i in range(floor(len(df_clean)/blockweite)):
      idx = blockweite * i
      block_df = df_clean[idx:idx+blockweite]
      # Hier die KI-Abfrage mit Protokoll-Funktion
      # Extra-Funktion, um Ausgabe auffangen zu können
      # Prüfe, ob korrektes JSON - wenn nein, versuch nochmal
      # (Kommt leider ab und zu vor, dass GPT3.5 kein korrektes JSON ausspuckt)
      no_valid_json = True
      attempts = 3
      while no_valid_json:
        json_str = test_block(block_df,
                            kat_text,
                            beispiele,
                            beispielantwort)
        try:
          t_df = json.loads(json_str)
        except:
          print("Kein gültiges JSON von der KI erzeugt - versuche es nochmal")
          attempts = attempts - 1
          if attempts <= 0:
              raise Exception("KI gibt ungültiges JSON zurück")
        else:
          try:
            t_df = rasterize(json_str)
          except:
            print("Rasterisierung scheitert, versuche es nochmal")
            attempts = attempts - 1
            if attempts <= 0:
                raise Exception("Rasterisierung gescheitert")
          else:
            # alles shiny
            no_valid_json = False
        # Fortschrittsbalken und Verbrauchsanzeige
        html_samples_tokens.value = f"Verbrauchte Tokens: {spent_tokens}, {spent_dollars:.2f} $"
      progress.value = idx + blockweite
      # Zum Anzeigen ein DF mit den Antworten
      t_df.index.name = "ID"
      # Braucht numerischen Index, NaNs abfischen und dann konvertieren
      t_df.index = t_df.index.astype('str')
      t_df.index = t_df.index.astype('int64')
      # Antworten und Kategorisierung vereinen
      tt_df = pd.merge(block_df,t_df,left_index=True,right_index=True)
      html_info.value=spinner_html + f"""<small>
      {tt_df.to_html()}</small>"""
      # Dataframe auswertung_df verlängern
      auswertung_df = pd.concat([auswertung_df,t_df])
      # z_dict zur Sicherheit mitführen
      if json.loads(json_str) != {}:
        z_dict.update(json.loads(json_str))

    # Alle Blöcke durch.
    # Fortschrittsbalken darf ins Ziel
    progress.value = idx + blockweite
    print("Auswertung abgeschlossen.")
    # Spinner und Tabellen-Infos löschen
    html_info.value = ""
    # Dateinamen erstellen, problematische Zeichen rausschmeißen
    sanitized = re.sub(r'[<>:"/\\|?*]', '', column_name)
    a_fname = "./ergebnisse/"+sanitized+".xlsx"
    # Die Antwort-Kategorien als Spaltennamen
    # Ersetzungstabelle bauen
    replacement_names = {f"{chr(n+ord('A'))}": labels[n].value for n in range(len(labels))}
    auswertung_df.rename(columns=replacement_names, inplace=True)
    auswertung_df.to_excel(a_fname, index=True)
    # Ergebnis exportieren
    html_info.value = f"""
    <h2>Ergebnistabelle fertig zum Download</h2>
    Erste fünf Zeilen: <br>
    {auswertung_df.head(5).to_html}
    Datei <code>{a_fname}</code> wird heruntergeladen
    """
    files.download(a_fname)
    print("Done.")

button_start_eval = widgets.Button(description = "Start",
                                   tooltip = "Starte Auswertung aller Antworten mit den Einstellungen")

button_start_eval.on_click(starte_evaluation)

display(dropdown_chk_model)
display(slider_blockweite)
display(widgets.Label("Letzte Möglichkeit, oben die Kategorien anzupassen!"))
display(button_start_eval)

In [ ]:
files.download(a_fname)

# V1.0

## Bugs und bekannte Probleme
- Checkt vor der Kategorisierung noch nicht das Kontextfenster!
- Manchmal kommen die Kategorien im falschen JSON-Format raus - das wird noch nicht abgefangen.
- Maximal 26 Kategorien - wird aber nicht überprüft

## Nice to have
- Vor-Kategorisierung der "grauen Kategorie" (nix zum Thema)

## Versionsgeschichte

### V1.0beta2, 24.2.2024
- Sicherheitsprüfung, falls weniger Antworten als Samples
- Funktioniert jetzt auch ohne Secret

### V1.0beta, 13.2.2024
- Excel-Dateien hochladen; Spalten auswerten
- Editor, um Kategorien und Einordnungs-Beispiele anzupassen
- Fortschrittsbalken
